In [1]:
%matplotlib inline 

import os
import json
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
eng_levels = [-2, -1, 0, 1, 2, 3]

## OpenPose Features

In this analysis, our aim is finding if we can get a relation between engagement and all sounds in the classroom.

In [3]:
basedir = "C:\\Users\\ASABUNCUOGLU13\\Documents\\data\\vol02\\slices-group\\frames-%d"
labeldir = "../scores/all_levels.csv"

In [4]:
labels = pd.read_csv(labeldir)

In [5]:
with open(os.path.join(basedir % 0, "keypoints", "f1_keypoints.json"), 'r') as f:
    data = json.load(f)
    p_id_range = range(len(data['people']))
    people_kpts = []
    for d in p_id_range:
        people_kpts.append(data['people'][d]['pose_keypoints_2d'])
    df = pd.DataFrame(np.array(people_kpts).flatten()).T
    df['label'] = labels['group'][0]

In [6]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,label
0,178.864,180.766,0.9683,198.379,261.046,0.854602,131.697,270.835,0.764647,125.874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [7]:
for i in range(21):
    for subdir, dirs, files in os.walk(os.path.join(basedir % i, "keypoints")):
        print(subdir, dirs)
        row = []
        for file in files:
            if file.endswith(".json"):
                with open(os.path.join(subdir, file), 'r') as f:
                    data = json.load(f)
                    p_id_range = range(len(data['people']))
                    people_kpts = []
                    for d in p_id_range:
                        people_kpts.append(data['people'][d]['pose_keypoints_2d'])
                    if(len(people_kpts) == 4):
                        df.loc[len(df)] = np.append(np.array(people_kpts).flatten(), labels['group'][len(df)-1])

C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-0\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-1\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-2\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-3\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-4\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-5\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-6\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-7\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-8\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-9\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-10\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data\vol02\slices-group\frames-11\keypoints []
C:\Users\ASABUNCUOGLU13\Documents\data

In [8]:
df.to_csv('features/pose_keypoints_with_labels.csv', index=False, header=True)
df.to_csv('features/pose_keypoints_with_labels.tsv', sep="\t", index=False, header=False)

In [ ]:
df['p_id'].to_csv('features/person_id.csv', index=False, header=True)
df['p_id'].to_csv('features/person_id.tsv', sep="\t", index=False, header=False)

In [5]:
df_pose = pd.read_csv('features/pose_keypoints_with_labels.csv')

In [9]:
def Euclidean_Dist(df1, df2):
    return np.linalg.norm(df1.values - df2.values, axis=1)

In [10]:
# get body keypoints

r_p1 = range(0,24)
r_p2 = range(75,99)
r_p3 = range(150, 174)
r_p4 = range(225,249)

df_p1 = df_pose.iloc[:, r_p1]
df_p2 = df_pose.iloc[:, r_p2]
df_p3 = df_pose.iloc[:, r_p3]
df_p4 = df_pose.iloc[:, r_p4]

In [16]:
#euclidean dist between body keypoints

e12 = Euclidean_Dist(df_p1, df_p2)
e13 = Euclidean_Dist(df_p1, df_p3)
e14 = Euclidean_Dist(df_p1, df_p4)
e23 = Euclidean_Dist(df_p2, df_p3)
e24 = Euclidean_Dist(df_p2, df_p4)
e34 = Euclidean_Dist(df_p3, df_p4)

In [21]:
np.transpose(e12)

array([1658.96218786, 1649.67493944, 1673.38693612, ...,  384.31475848,
        383.24986902,  843.55736258])

In [23]:
pd.DataFrame(np.transpose([e12, e12, e14, e23, e24, e34]), columns=["e12", "e13", "e14", "e23", "e24", "e34"]).to_csv("features/pose_distances.csv", index=False, header=True)